# Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.


In [1]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\acer\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2 as cv
import numpy as np
import imutils
import matplotlib.pyplot as plt
from IPython.display import Image

In [3]:
# Question1
image = cv.imread('images/lena.jfif')

rotated = imutils.rotate_bound(image, angle = 45)

cv.imshow('Rotated Image with imutils', rotated)
cv.imwrite('images/lena_imutils_rotate_45.png', rotated) 

rows, cols = image.shape[:2]

# Define the rotation angle 
angle = 45
angle_rad = np.deg2rad(angle)

# Calculate the new image dimensions after rotation
new_width = int(cols * np.abs(np.cos(angle_rad)) + rows * np.abs(np.sin(angle_rad)))
new_height = int(rows * np.abs(np.cos(angle_rad)) + cols * np.abs(np.sin(angle_rad)))

# Compute the rotation matrix
M = cv.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)

# Adjust the translation component of the matrix to avoid cropping
M[0, 2] += (new_width - cols) / 2
M[1, 2] += (new_height - rows) / 2

# Perform the rotation
rotated = cv.warpAffine(image, M, (new_width, new_height), borderValue=(255, 255, 255))

# Display the rotated image
cv.imshow('Rotated Image with numpy', rotated)
cv.imwrite('images/lena_imutils_rotate_45.png', image) 
cv.waitKey(0)
cv.destroyAllWindows()

2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")

In [6]:
# Question2
bee = cv.imread("images/native-bee.png")
flower = cv.imread("images/flower.jfif")

# get dimension of flower img
rows, cols, cahnnels = flower.shape
# define roi for flower
roi = bee[0:rows, 0:cols]

# convert flower image to grayscale
flower_gray = cv.cvtColor(flower, cv.COLOR_BGR2GRAY)    

# create binary mask
ret, mask = cv.threshold(flower_gray, 70, 255, cv.THRESH_BINARY)     
mask_inv = cv.bitwise_not(mask)    # create inverse mask

bee_bg = cv.bitwise_and(roi, roi, mask = mask_inv)     # black out area of flower in bee
flower_fg = cv.bitwise_and(flower, flower, mask = mask)    # take region of flower

# add flower to roi & modify main image
dst = cv.add(bee_bg, flower_fg)
bee[0:rows, 0:cols] = dst

cv.imshow('result',bee)
cv.imwrite("images/flower_bee.png",bee)
cv.waitKey(0)
cv.destroyAllWindows()

3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```


In [14]:
def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    """"""
    
    height, width = image.shape[:2]

    # Calculate crop sizes based on ratios
    min_crop_size = int(min(height, width) * min_crop_ratio)
    max_crop_size = int(min(height, width) * max_crop_ratio)

    # Randomly select crop size
    crop_size = np.random.randint(min_crop_size, max_crop_size + 1)

    # Calculate crop region
    x = (width - crop_size) // 2
    y = (height - crop_size) // 2

    # Perform cropping
    img_crop = image[y:y+crop_size, x:x+crop_size]

    return img_crop

In [15]:
# Read the image
img = cv.imread("images/car.jpg")
img_crop = random_center_crop(img, 0.5, 0.8)

cv.imshow('result', img_crop)
cv.imwrite("images/img_crop.png", img_crop)
cv.waitKey(0)
cv.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'


4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [17]:
image = cv.imread('images/native-bee.png', cv.IMREAD_COLOR)

# Generate Poisson noise
noisy_image = np.random.poisson(image / 255.0 * 100) / 100.0 * 255.0

# Ensure values are within valid range
noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

# Display original and noisy image
cv.imshow('Original Image', image)
cv.imshow('Noisy Image (Poisson)', noisy_image)
cv.imwrite('images/noisy_bee.png',noisy_image)
cv.waitKey(0)
cv.destroyAllWindows()